## **Notebook com os Modelos finais**

**Autores:**

- Arthur Brandão do Nascimento

- Caio Ávila Paulo

- Matheus Macedo do Nascimento

## **Introdução**
Este notebook tem por objetivo implementar e comparar diferentes algoritmos de aprendizado de máquina supervisionado de classificação. A partir de características como composição, tamanho e tempo de exposição de diferentes materiais, será prevista sua toxicidade.

Os algoritmos utilizados serão os de k vizinhos mais próximos (knn), árvore de decisão, floresta aleatória, Support Vector Classifier (SVC) e Regressão Logística. O desempenho de cada um deles será estimado por validação cruzada do tipo k-fold e os hiperparâmetros dos modelos serão otimizados com o ``optuna``. Além disso, um modelo baseline será estabelecido para fins de comparação.

### **Importando as bibliotecas necessárias**

In [52]:
import numpy as np
import pandas as pd
import sklearn as sk
from optuna import create_study
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MaxAbsScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from scipy import stats

### **Importando o dataset**

O dataset escolhido tem, a princípio 3923 linhas e 17 colunas. 

In [53]:
df = pd.read_csv("../datasets/dataset_nanotoxicologia_combinado.csv")

display(df.shape)
display(df.head())

(3923, 17)

,Material_type,Core_size,Hydro_size,Surface_charge,Surface_area,Formation_enthalpy,Conduction_band,Valence_band,Electronegativity,Assay,Cell_name,Cell_species,Cell_origin,Cell_type,Exposure_time,Exposure_dose,Toxicity
0,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,MTT,HCMEC,Human,Blood,Normal,24.0,0.001,Nontoxic
1,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,MTT,HCMEC,Human,Blood,Normal,24.0,0.010,Nontoxic
2,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,MTT,HCMEC,Human,Blood,Normal,24.0,0.100,Nontoxic
3,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,MTT,HCMEC,Human,Blood,Normal,24.0,1.000,Nontoxic
4,Al2O3,39.7,267.0,36.3,64.7,-17.345,-1.51,-9.81,5.67,MTT,HCMEC,Human,Blood,Normal,24.0,5.000,Nontoxic


### **Definindo as ``FEATURES`` e o ``TARGET``**

As features serão separadas entre aquelas que já são valores numéricos (``FEATURES_NUM``) e aquelas que serão convertidas em valores binários (``FEATURES_DUMMY``).

Variável alvo (**``TARGET``**):
- ``"Toxicity"`` - 

Features Numéricas (**``FEATURES_NUM``**):

[Adicionar o que é cada uma das colunas do dataset]

In [54]:
FEATURES_NUM = ["Core_size", 
                "Hydro_size", 
                "Surface_charge", 
                "Surface_area", 
                "Formation_enthalpy", 
                "Conduction_band", 
                "Valence_band", 
                "Electronegativity", 
                "Exposure_time", 
                "Exposure_dose"
]

FEATURES_DUMMY = ["Material_type", "Assay", "Cell_name", "Cell_species", "Cell_origin", "Cell_type"]

TARGET = ["Toxicity"]

### **Evitando vazamento de dados pelo ``groupby()``**
Pode ser que vários dados do dataset sejam iguais em todos os atributos, diferindo (ou não) apenas no target. Dessa forma, alguns desses dados poderiam acabar sendo usado na etapa de treino e outros na fase de teste. Isso faria com que a métrica não refletisse o real desempenho do modelo; afinal, ele já "conheceria" alguns dados de teste, o que acarreta uma previsão enviesada.

Para evitar esse tipo de vazamento, *antes* do split de treino e teste, é necessário acabar com essa redundância. Isso é feito agrupando todos os dados duplicados em um só: os atributos continuam os mesmos, mas apenas um valor de target é utilizado, a partir de uma estatística dos dados originais. Como o target é categórico, será usada a moda.

Importante que os dados duplicados não precisam ser cópias idênticas: se todos os atributos forem muito próximos (apesar de não serem iguais), o vazamento de dados ocorrerá da mesma maneira. Por isso, antes de identificar dados repetidos e agrupá-los, arredonda-se o valor de cada atributo em uma casa adequada.

#### **Arredondando**

In [55]:
casa_arredondamento = {
    "Core_size": 0, 
    "Hydro_size": 0, 
    "Surface_charge": 0, 
    "Surface_area": 0, 
    "Formation_enthalpy": 0, 
    "Conduction_band": 0,
    "Valence_band": 0, 
    "Electronegativity": 0, 
    "Exposure_time": 1, 
    "Exposure_dose": 1,
}

df_round = df.round(casa_arredondamento)

df_round.shape

(3923, 17)

Como nós podemos ver pelo código abaixo esse dataset possui alguns dados duplicados

In [56]:
num_duplicados = df_round[FEATURES_NUM + FEATURES_DUMMY].duplicated().sum()
print(f"Número de linhas duplicadas em df_round: {num_duplicados}")

Número de linhas duplicadas em df_round: 43


#### **Agrupando**

O [``agg()``](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html) é um método do pandas usado para aplicar uma ou mais operações de agregação em grupos de dados. Como o target (``"Toxicity"``) é categórico, usaremos a moda para decidir o rótulo daquele grupo.

In [57]:
from scipy import stats

def calcular_moda(serie):
    """Calcula a moda de uma série, retornando o primeiro valor se não houver moda clara"""
    moda = stats.mode(serie)
    return moda[0]

In [58]:
df_grouped = df_round.groupby(FEATURES_NUM + FEATURES_DUMMY, sort=False)

def calcular_moda(serie):
    """Calcula a moda de uma série"""
    moda = serie.mode()
    return moda[0]

agg_dict = {
    **{col: "mean" for col in FEATURES_NUM},
    **{col: calcular_moda for col in FEATURES_DUMMY},
    "Toxicity": calcular_moda
}

df_grouped = df_grouped.agg(agg_dict)
df_grouped = df_grouped.reset_index(drop=True)

print(f"O shape (linhas x colunas) do df_round é: {df_round.shape}")
print(f"O shape (linhas x colunas) do df_tratado é: {df_grouped.shape}")

O shape (linhas x colunas) do df_round é: (3923, 17)
O shape (linhas x colunas) do df_tratado é: (3880, 17)


In [59]:
num_duplicados = df_grouped[FEATURES_NUM + FEATURES_DUMMY].duplicated().sum()
print(f"Número de linhas duplicadas em df_grouped: {num_duplicados}")

Número de linhas duplicadas em df_grouped: 0


Como podemos ver não há mais valores duplicados

### **Fazendo a codificação One-Hot**
Os algoritmos de aprendizado de máquina utilizados exigem que todos os atributos sejam numéricos. Dessa forma, é necessário transformar os dados qualitativos adequadamente.  O codificador One-Hot transforma uma coluna de dados categóricos em várias colunas, cada qual representando um dos rótulos possíveis; se o dado originalmente tinha aquele rótulo, atribui-se o valor 1, caso contrário preenche-se com 0.

In [60]:
encoder = OneHotEncoder(sparse_output=False, dtype=np.int32)
dummy_encoded = encoder.fit_transform(df_grouped[FEATURES_DUMMY])

dummy_columns = encoder.get_feature_names_out(FEATURES_DUMMY)
df_dummy = pd.DataFrame(dummy_encoded, columns=dummy_columns, index=df_grouped.index)

df_dummy = pd.concat([df_grouped[FEATURES_NUM + TARGET], df_dummy], axis=1)
FEATURES_FINAL = FEATURES_NUM + list(dummy_columns)

display(df_dummy.shape)
display(df_dummy.head(5))

(3880, 203)

,Core_size,Hydro_size,Surface_charge,Surface_area,Formation_enthalpy,Conduction_band,Valence_band,Electronegativity,Exposure_time,Exposure_dose,...,Cell_origin_Pancreas,Cell_origin_Plant cell,Cell_origin_Prostate,Cell_origin_Skin,Cell_origin_Stomach,Cell_origin_Tetis,Cell_origin_Tongue,Cell_origin_Umbilical vein,Cell_type_Cancer,Cell_type_Normal
0,40.0,267.0,36.0,65.0,-17.0,-2.0,-10.0,6.0,24.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,40.0,267.0,36.0,65.0,-17.0,-2.0,-10.0,6.0,24.0,0.1,...,0,0,0,0,0,0,0,0,0,1
2,40.0,267.0,36.0,65.0,-17.0,-2.0,-10.0,6.0,24.0,1.0,...,0,0,0,0,0,0,0,0,0,1
3,40.0,267.0,36.0,65.0,-17.0,-2.0,-10.0,6.0,24.0,5.0,...,0,0,0,0,0,0,0,0,0,1
4,40.0,267.0,36.0,65.0,-17.0,-2.0,-10.0,6.0,24.0,10.0,...,0,0,0,0,0,0,0,0,0,1


### **Definindo os dados de treino e de teste**
Com o dataframe devidamente tratado, pode ser feita a divisão dos dados em teste e treino. No caso será utilizado o ``stratify`` pois há um certo desbalanço no dataset, havendo mais dados sobre nanopartículas não tóxicas do que nanopartículas tóxicas, o ``"stratify"`` mantém a proporção das classes em ambos os conjuntos.

In [61]:
TAMANHO_TESTE = 0.25
SEED = 404

valores_target = df_dummy[TARGET].values.ravel()

df_treino, df_teste = train_test_split(df_dummy, test_size=TAMANHO_TESTE, random_state=SEED, stratify=valores_target)

X_teste = df_teste.reindex(FEATURES_FINAL, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()

X_treino = df_treino.reindex(FEATURES_FINAL, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()

### **Criando os modelos e espaços de busca**

Serão criados os seguintes modelos para comparação:

- Baseline (DummyClassifier)
- k-Nearest Neighbors classifier (KNN)
- Árvore de Decisão
- Floresta Aleatória
- Regressão Logística
- Support Vector Classifier


#### **Baseline**

O baseline estabelece uma referência mínima de desempenho, onde qualquer modelo útil deve superar essa referência.

No caso, foi utilizado o ``DummyClassifier(strategy='most_frequent')``, que prevê sempre a moda dos valores de y.

In [62]:
modelo_baseline = DummyClassifier(strategy="most_frequent")

f1_macro_estimativa_dummy = cross_val_score(modelo_baseline, X_treino, y_treino, scoring="f1_macro", cv=10).mean()

#### **Implementação dos Modelos com o Optuna**
O Optuna é um framework de otimização de hiperparâmetros. Ele automatiza o processo de enontrar o conjunto ótimo de hiperparâmetros para um dado modelo, almeando minimizar ou maximizar uma função objetiva específica.

No Optuna, os ``Trials`` são as tentativas com diferentes combinações de hiperparâmetros e o ``Study`` é o conjunto de trials para um determinado objetivo.

Nesse contexto, a função ``cria_instancia_modelo`` serve para criar uma instância do modelo escolhido, recebendo um trial.
Para definir o espaço do dicionário dos parâmetros é usado o ``trial.suggest_*()``.

Além disso, foi utilizada a decisão de normalização dos dados como um hiperparâmetro adicional. Para isso foi criado um curto **pipeline** com o ``make_pipeline()``

#### **Instância K-NN**

Baseado no princípio de que exemplos similares tendem a pertencer à mesma classe. Para classificar uma nova amostra, o algoritmo calcula as distâncias entre essa amostra e os pontos do conjunto de treinamento, identifica os K vizinhos mais próximos e realiza uma votação majoritária entre suas classes.

Hiperparâmetros Otimizados:

- **``n_neighbors``**: número de vizinhos

- **``weights``**: uniform, sem pesos, ou distance, em que vizinhos mais próximos tem mais peso

- **``p``**: tipo de distância, 1=Manhattan, 2=Euclidiana

In [63]:
def cria_instancia_knn(trial):
    """Cria uma instância de um modelo KNN."""

    parametros = {
        "n_neighbors": trial.suggest_int("num_vizinhos", 1, 200, log=True),
        "weights": trial.suggest_categorical("pesos", ["uniform", "distance"]),
        "p": trial.suggest_int("tipo_distancia", 1, 2),
        "n_jobs": -1,
    }

    normalizar = trial.suggest_categorical("normalizar", [True, False])

    if normalizar:
        tipo_normalizacao = trial.suggest_categorical("tipo_norm", ["Standard", "MinMax", "MaxAbs"])

        if tipo_normalizacao == "Standard":
            normalizador = StandardScaler()
        elif tipo_normalizacao == "MinMax":
            normalizador = MinMaxScaler()
        elif tipo_normalizacao == "MaxAbs":
            normalizador = MaxAbsScaler()
            
        modelo_knn = make_pipeline(
            normalizador,
            KNeighborsClassifier(**parametros)
        )
    
    else:
        modelo_knn = KNeighborsClassifier(**parametros)

    return modelo_knn

#### **Instância Árvore de Decisão**

O algoritmo da árvore de decisão constrói uma estrutura similar a um fluxograma, onde cada nó interno representa uma decisão baseada em uma feature específica, cada ramo representa o resultado dessa decisão e cada nó folha representa a classe predita. O processo de construção da árvore segue uma estratégia visa a máxima separação dos nós resultantes em cada divisão, utilizando critérios como entropia ou índice Gini para medir a homogeneidade das classes. A principal vantagem das árvores de decisão reside em sua alta interpretabilidade.

Hiperparâmetros Otimizados:

- **``max_depth``**: Profundidade máxima da árvore

- **``criterion``**: A função usada para medir a qualidade de um _split_ em cada nó

- **``min_samples_split``**: Número mínimo de amostras necessárias para dividir um nó

- **``min_samples_leaf``**:  Número mínimo de amostras em um nó folha

- **``max_features``**: Limita o número de features que o algorítimo utiliza em cada divisão para determinar a melhor features de divisão.


In [64]:
def cria_instancia_dtree(trial):
    """Cria a instância de um modelo de árvore de decisão"""

    parametros = {
        "max_depth": trial.suggest_int("profundidade", 2, 600, log=True),
        "criterion": trial.suggest_categorical("critério", ['entropy', 'log_loss', 'gini']),
        "min_samples_split": trial.suggest_int("min_exemplos_split", 2, 200, log=True),
        "min_samples_leaf": trial.suggest_int("min_exemplos_folha", 1, 100, log=True),
        "max_features": trial.suggest_float("num_max_features", 0, 1),
        "random_state": SEED,
    }

    normalizar = trial.suggest_categorical("normalizar", [True, False])

    if normalizar:
        tipo_normalizacao = trial.suggest_categorical("tipo_norm", ["Standard", "MinMax", "MaxAbs"])

        if tipo_normalizacao == "Standard":
            normalizador = StandardScaler()
        elif tipo_normalizacao == "MinMax":
            normalizador = MinMaxScaler()
        elif tipo_normalizacao == "MaxAbs":
            normalizador = MaxAbsScaler()
            
        modelo_dtree = make_pipeline(
            normalizador,
            DecisionTreeClassifier(**parametros)
        )
    
    else:
        modelo_dtree = DecisionTreeClassifier(**parametros)

    return modelo_dtree

#### **Instância Floresta Aleatória (RF)**

A floresta aleatória é uma técnica que combina o princípio de comitês com aleatorização adicional de features para criar múltiplas árvores de decisão diversas. Cada árvore na floresta é treinada em uma amostra bootstrap, amostragem aleatória com reposição, do conjunto de dados original e, em cada divisão da árvore, apenas um subconjunto aleatório de features é considerado para divisão. Essa aleatorização garante que as árvores individuais sejam diferentes umas das outras, reduzindo a correlação entre seus erros. Durante a predição, todas as árvores votam na classe final, com a maioria decidindo o resultado. Essa abordagem coletiva resulta em um modelo que geralmente supera árvores individuais, embora possua uma interpretabilidade muito inferior.

Hiperparâmetros Otimizados:

- **``n_estimators``**: Número de árvores utilizadas no modelo

- **``criterion``**: A função usada para medir a qualidade de um _split_ em cada nó

- **``max_depth``**: Controla profundidade maxíma individual das árvores

- **``min_samples_split``**: Número mínimo de amostras necessárias para dividir um nó

- **``min_samples_leaf``**: Número mínimo de amostras em um nó folha

- **``max_features``**: Limita o número de features que o algorítimo utiliza em cada divisão para determinar a melhor features de divisão.

In [65]:
def cria_instancia_rf(trial):
    """Cria a instância de um modelo de floresta aleatória."""

    parametros = {
        "n_estimators": trial.suggest_int("num_arvores", 2, 1000, log=True),
        "criterion": trial.suggest_categorical("critério", ["log_loss", "gini", "entropy"]),
        "max_depth": trial.suggest_int("max_depth", 1, 600, log=True),
        "min_samples_split": trial.suggest_int("min_exemplos_split", 2, 200),
        "min_samples_leaf": trial.suggest_int("min_exemplos_folha", 1, 100),
        "max_features": trial.suggest_float("num_max_atributos", 0, 1),
        "n_jobs": -1,
        "bootstrap": True,
        "random_state": SEED,
    }

    normalizar = trial.suggest_categorical("normalizar", [True, False])
    if normalizar:
        tipo_normalizacao = trial.suggest_categorical("tipo_norm", ["Standard", "MinMax", "MaxAbs"])

        if tipo_normalizacao == "Standard":
            normalizador = StandardScaler()
        elif tipo_normalizacao == "MinMax":
            normalizador = MinMaxScaler()
        elif tipo_normalizacao == "MaxAbs":
            normalizador = MaxAbsScaler()

        modelo_rf = make_pipeline(
            normalizador,
            RandomForestClassifier(**parametros)
        )
    
    else:
        modelo_rf = RandomForestClassifier(**parametros)

    return modelo_rf

#### **Instância Regressão Logística (LR)**

A regressão logística é um algoritmo de classificação que modela a probabilidade de uma amostra pertencer a uma classe específica usando a função logística (sigmoide). O modelo assume uma relação linear entre as features e o logarítimo da chance da probabilidade da classe positiva. A inclusão de termos de regularização (L1, L2 ou Elastic Net) ajuda a prevenir overfitting e a lidar com multicolinearidade entre as features.

Hiperparâmetros Otimizados:

- **``penalty``**: É o tipo de regularização utilizada 

- **``C``**: Controla a intesidade da regularização

- **``l1_ratio ``**: Balanceamento entre penalidades L1 e L2, utilizada apenas pelo elasticnet

Há também um parâmetro chamado `class_weight` que nos auxilia a lidar com variáveis ditas "não-balanceadas", basicamente esse hiperparâmetro decide se o modelo vai se importar mais com a acurácia no geral ou na identificação correta de _outliers_. Ele ajusta a influência de cada uma das clsses durante o treinamento, forçando o modelo a prestar mais atenção nas classes minoritárias.

In [66]:
def cria_instancia_lr(trial):
    """Cria a instância de um modelo de Regressão Logística."""

    parametros = {
        "penalty": trial.suggest_categorical("penalidade", ["l1", "l2", "elasticnet"]),
        "C": trial.suggest_float("C", 1e-3, 1e3, log=True),
        "class_weight": "balanced", 
        "solver": "saga",
        "max_iter": 10000,
        "n_jobs": -1,
        "random_state": SEED,
    }

    if parametros["penalty"] == "elasticnet":
        parametros["l1_ratio"] = trial.suggest_float("l1_ratio", 0.1, 0.9)
    
    normalizar = trial.suggest_categorical("normalizar", [True, False])

    if normalizar:
        tipo_normalizacao = trial.suggest_categorical("tipo_norm", ["Standard", "MinMax", "MaxAbs"])

        if tipo_normalizacao == "Standard":
            normalizador = StandardScaler()
        elif tipo_normalizacao == "MinMax":
            normalizador = MinMaxScaler()
        elif tipo_normalizacao == "MaxAbs":
            normalizador = MaxAbsScaler()
            
        modelo_lr = make_pipeline(
            normalizador,
            LogisticRegression(**parametros)
        )
    
    else:
        modelo_lr = LogisticRegression(**parametros)

    return modelo_lr

#### **Instância Support Vector Classifier (SVC)**

O SVC é um algoritimo que tem uma fundamentação matemática centrada em encontrar hiperplanos que maximizem as chamadas "margens" (distância entre o hiperplano de decisão e os exemplos mais próximos de cada classe) entre classes. Os hiperparâmetros controlam coisas como: o quão estrita a separação de pontos deve ser (hiperparâmetro c), que tipo de barreira de decisão usar (kernel) e o quão complexa essa barreira pode ser (gamma, degree). É vital otimizá-los para poder obter um modelo funcional que não superestime ou subestime a contribuição de uma variável.

Hiperparâmetros Otimizados:

- **``C``**: é um parâmetro regularizador, que controla a troca entre tamanho da margem e erro de classificação, um C pequeno tem melhor generalização, já um C maior tem um risco de _overfitting_.

- **``kernel``**: controla o formato da fronteira de decisão.

- **``gamma``**: é o coeficiente do kernel, que influencia o alcance de cada ponto individual no treinamento.

- **``degree``**: é grau polinomial, caso o kernel seja polinomial, controla a complexidade do polinômio.

- **``coef0``**: é o termo independente, que controla o deslocamento em kernels polinomiais ou sigmóides

Há também um parâmetro chamado `class_weight` que nos auxilia a lidar com variáveis ditas "não-balanceadas", basicamente esse hiperparâmetro decide se o modelo vai se importar mais com a acurácia no geral ou na identificação correta de _outliers_. Ele ajusta a influência de cada uma das clsses durante o treinamento, forçando o modelo a prestar mais atenção nas classes minoritárias.

In [67]:
def cria_instancia_svc(trial):
    """Cria a instância de um modelo de SVC"""
    
    parametros = {
        "C": trial.suggest_float("C", 0.01, 1000.0, log=True),
        "kernel": trial.suggest_categorical("kernel", ["linear", "rbf","poly", "sigmoid"]),
        "class_weight": "balanced",
        "random_state": SEED,
        "max_iter": 600000,
    }

    if parametros["kernel"] in ["rbf", "poly", "sigmoid"]:
        entrada_gamma = trial.suggest_categorical("entrada_gamma", ["scale", "auto", "float"])

        if entrada_gamma == "scale":
            parametros["gamma"] = "scale"
        elif entrada_gamma == "auto":
            parametros["gamma"] = "auto"
        elif entrada_gamma == "float":
            parametros["gamma"] = trial.suggest_float("gamma", 1e-5, 1e2, log=True)
    
    if parametros["kernel"] == "poly":
        parametros["degree"] = trial.suggest_int("degree", 1, 4)

    normalizar = trial.suggest_categorical("normalizar", [True, False])

    if normalizar:
        tipo_normalizacao = trial.suggest_categorical("tipo_norm", ["Standard", "MinMax", "MaxAbs"])

        if tipo_normalizacao == "Standard":
            normalizador = StandardScaler()
        elif tipo_normalizacao == "MinMax":
            normalizador = MinMaxScaler()
        elif tipo_normalizacao == "MaxAbs":
            normalizador = MaxAbsScaler()
            
        modelo_svc = make_pipeline(
            normalizador,
            SVC(**parametros)
        )
    
    else:
        modelo_svc = SVC(**parametros)

    return modelo_svc

### **Função Objetivo**

A seguir foi criada a função objetivo que é a função que irá computar a métrica de interesse. Neste caso, a métrica de interesse é a f1-macro, obtida por validação cruzada.

A métrica ``f1-macro`` funciona da seguinte forma:

- Precisão = Verdadeiros Positivos / (Verdadeiros Positivos + Falsos Positivos) 
    - De todas as previsões positivas, quantas eram realmente positivas

- Recall = Verdadeiros Positivos / (Verdadeiros Positivos + Falsos Negativos) 
    - De todos os casos realmente positivos, quantos foram identificados corretamente

A métrica calcula a média harmonica entre a preicisão e o recall:

- **F1 = 2 × (Precision × Recall) / (Precision + Recall)**

Nela, cada classe tem o mesmo peso, independente de quantos exemplos tenha, o que é útil para o nosso caso.

In [68]:
def funcao_objetivo(trial, X, y, num_folds, modelo="knn"):
    """Função objetivo do optuna"""

    if modelo == "knn":
        modelo = cria_instancia_knn(trial)
    elif modelo == "dtree":
        modelo = cria_instancia_dtree(trial)
    elif modelo == "rf":
        modelo = cria_instancia_rf(trial)
    elif modelo == "lr":
        modelo = cria_instancia_lr(trial)
    elif modelo == "svc":
        modelo = cria_instancia_svc(trial)
    
    metricas = cross_val_score(
        modelo, 
        X, 
        y, 
        scoring="f1_macro",
        cv=num_folds,
        )
    
    return metricas.mean()


### **Otimizando os Hiperparâmetros**

A seguir foram criados os estudos (conjunto de trials) usando o ``create_study()``, cujo o argumento ``direction='maximizar'`` tem como objetivo minimizar a [métrica]. Foi o utilizado o ``storage`` para armazenar o progreso da busca e o  ``load_if_exists`` para que seja possível continuar a busca de onde ela parou.

In [69]:
NOME_DO_ESTUDO_KNN = "knn_nanotoxiclogia_optuna"
NOME_DO_ESTUDO_DTREE = "dtree_nanotoxiclogia_optuna"
NOME_DO_ESTUDO_SVC = "svc_nanotoxiclogia_optuna"
NOME_DO_ESTUDO_RF = "rf_nanotoxiclogia_optuna"
NOME_DO_ESTUDO_LR = "lr_nanotoxiclogia_optuna"

objeto_de_estudo_knn = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO_KNN,
    storage=f"sqlite:///../resultados_optuna/{NOME_DO_ESTUDO_KNN}.db",
    load_if_exists=True,
)

objeto_de_estudo_dtree = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO_DTREE,
    storage=f"sqlite:///../resultados_optuna/{NOME_DO_ESTUDO_DTREE}.db",
    load_if_exists=True,
)

objeto_de_estudo_svc = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO_SVC,
    storage=f"sqlite:///../resultados_optuna/{NOME_DO_ESTUDO_SVC}.db",
    load_if_exists=True,
)

objeto_de_estudo_rf = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO_RF,
    storage=f"sqlite:///../resultados_optuna/{NOME_DO_ESTUDO_RF}.db",
    load_if_exists=True,
)

objeto_de_estudo_lr = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO_LR,
    storage=f"sqlite:///../resultados_optuna/{NOME_DO_ESTUDO_LR}.db",
    load_if_exists=True,
)

[I 2025-11-03 09:16:04,424] Using an existing study with name 'knn_nanotoxiclogia_optuna' instead of creating a new one.
[I 2025-11-03 09:16:04,469] Using an existing study with name 'dtree_nanotoxiclogia_optuna' instead of creating a new one.
[I 2025-11-03 09:16:04,677] Using an existing study with name 'svc_nanotoxiclogia_optuna' instead of creating a new one.
[I 2025-11-03 09:16:04,714] Using an existing study with name 'rf_nanotoxiclogia_optuna' instead of creating a new one.
[I 2025-11-03 09:16:04,747] Using an existing study with name 'lr_nanotoxiclogia_optuna' instead of creating a new one.


Para realmente rodar o otimizador precisamos de uma função objetivo que tenha apenas um argumento, o `trial`. Para isso vamos definir a `funcao_objetivo_parcial`.

Serão usados ``modelo='modelo'`` para cada modelo e um número de fols na validação cruzada igual a 10.

In [70]:
NUM_FOLDS = 10

def funcao_objetivo_parcial_knn(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS, modelo="knn")

def funcao_objetivo_parcial_dtree(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS, modelo="dtree")

def funcao_objetivo_parcial_rf(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS, modelo="rf")

def funcao_objetivo_parcial_svc(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS, modelo="svc")

def funcao_objetivo_parcial_lr(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS, modelo="lr")


Agora podemos definiro o número de novos trials e rodar cada otimização de modelo separadamente.

In [71]:
NUM_TENTATIVAS = 1
objeto_de_estudo_knn.optimize(funcao_objetivo_parcial_knn, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 09:16:08,519] Trial 1014 finished with value: 0.8402578592356329 and parameters: {'num_vizinhos': 1, 'pesos': 'uniform', 'tipo_distancia': 1, 'normalizar': True, 'tipo_norm': 'MaxAbs'}. Best is trial 148 with value: 0.8402578592356329.


In [72]:
NUM_TENTATIVAS = 1
objeto_de_estudo_dtree.optimize(funcao_objetivo_parcial_dtree, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 09:16:09,280] Trial 1011 finished with value: 0.7552768782478383 and parameters: {'profundidade': 378, 'critério': 'entropy', 'min_exemplos_split': 2, 'min_exemplos_folha': 83, 'num_max_features': 0.5453290174242155, 'normalizar': False}. Best is trial 781 with value: 0.8961577595288454.


In [73]:
NUM_TENTATIVAS = 1
objeto_de_estudo_svc.optimize(funcao_objetivo_parcial_svc, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 09:16:12,082] Trial 1412 finished with value: 0.8363184066773817 and parameters: {'C': 605.2164052576776, 'kernel': 'rbf', 'entrada_gamma': 'float', 'gamma': 0.0006509292159462127, 'normalizar': False}. Best is trial 1059 with value: 0.8508233453451532.


In [74]:
NUM_TENTATIVAS = 1
objeto_de_estudo_rf.optimize(funcao_objetivo_parcial_rf, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 09:16:22,283] Trial 1418 finished with value: 0.8510409801370791 and parameters: {'num_arvores': 572, 'critério': 'log_loss', 'max_depth': 189, 'min_exemplos_split': 13, 'min_exemplos_folha': 5, 'num_max_atributos': 0.12432683735393313, 'normalizar': False}. Best is trial 898 with value: 0.9112519576543396.


In [ ]:
NUM_TENTATIVAS = 0
objeto_de_estudo_lr.optimize(funcao_objetivo_parcial_lr, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 09:16:39,980] Trial 534 finished with value: 0.7361347729403225 and parameters: {'penalidade': 'elasticnet', 'C': 198.09079207236158, 'l1_ratio': 0.3949838695826708, 'normalizar': True, 'tipo_norm': 'MinMax'}. Best is trial 12 with value: 0.7383435156883571.


### **Vizualizando os Resultados**

Agora podemos analizar o foi obtido com cada modelo e a partir disso definir qual deles teve o melhor estimativa de resultado.

#### **Resultado Dummy (baseline):**

In [76]:
print(f"A estimativa do f1-macro para o Dummy foi: {f1_macro_estimativa_dummy}")

A estimativa do f1-macro para o Dummy foi: 0.4339620717011069


#### **Resultado K-NN:**

In [77]:
melhor_trial_knn = objeto_de_estudo_knn.best_trial

print(f"Número do melhor trial K-NN: {melhor_trial_knn.number}")
print(f"Parâmetros do melhor trial : {melhor_trial_knn.params}")
print(f"A melhor estimativa do f1-macro para o K-NN foi: {objeto_de_estudo_knn.best_value}")

Número do melhor trial K-NN: 148
Parâmetros do melhor trial : {'num_vizinhos': 1, 'pesos': 'uniform', 'tipo_distancia': 1, 'normalizar': True, 'tipo_norm': 'MaxAbs'}
A melhor estimativa do f1-macro para o K-NN foi: 0.8402578592356329


#### **Resultado Árvore de Decisão:**

In [78]:
melhor_trial_dtree = objeto_de_estudo_dtree.best_trial

print(f"Número do melhor trial da Árvore de Decisão: {melhor_trial_dtree.number}")
print(f"Parâmetros do melhor trial da Árvore de Decisão: {melhor_trial_dtree.params}")
print(f"A melhor estimativa do f1-macro para a Árvore de Decisão foi: {objeto_de_estudo_dtree.best_value}")

Número do melhor trial da Árvore de Decisão: 781
Parâmetros do melhor trial da Árvore de Decisão: {'profundidade': 29, 'critério': 'entropy', 'min_exemplos_split': 2, 'min_exemplos_folha': 1, 'num_max_features': 0.4819347463126588, 'normalizar': True, 'tipo_norm': 'MinMax'}
A melhor estimativa do f1-macro para a Árvore de Decisão foi: 0.8961577595288454


#### **Resultado SVC:**

In [79]:
melhor_trial_svc = objeto_de_estudo_svc.best_trial

print(f"Número do melhor trial do SVC: {melhor_trial_svc.number}")
print(f"Parâmetros do melhor trial do SVC: {melhor_trial_svc.params}")
print(f"A melhor estimativa do f1-macro para o SVC foi: {objeto_de_estudo_svc.best_value}")

Número do melhor trial do SVC: 1059
Parâmetros do melhor trial do SVC: {'C': 992.1950516156445, 'kernel': 'rbf', 'entrada_gamma': 'float', 'gamma': 3.25151263422672e-05, 'normalizar': False}
A melhor estimativa do f1-macro para o SVC foi: 0.8508233453451532


#### **Resultado Floresta Aleatória:**

In [80]:
melhor_trial_rf = objeto_de_estudo_rf.best_trial

print(f"Número do melhor trial da Floresta Aleatória: {melhor_trial_rf.number}")
print(f"Parâmetros do melhor trial da Floresta Aleatória: {melhor_trial_rf.params}")
print(f"A melhor estimativa do f1-macro para a Floresta Aletória foi: {objeto_de_estudo_rf.best_value}")

Número do melhor trial da Floresta Aleatória: 898
Parâmetros do melhor trial da Floresta Aleatória: {'num_arvores': 403, 'critério': 'entropy', 'max_depth': 301, 'min_exemplos_split': 6, 'min_exemplos_folha': 1, 'num_max_atributos': 0.5282007906767744, 'normalizar': False}
A melhor estimativa do f1-macro para a Floresta Aletória foi: 0.9112519576543396


#### **Resultado Regressão Logística:**

In [81]:
melhor_trial_lr = objeto_de_estudo_lr.best_trial

print(f"Número do melhor trial da Regressão Logística: {melhor_trial_lr.number}")
print(f"Parâmetros do melhor trial da Regressão Logística: {melhor_trial_lr.params}")
print(f"A melhor estimativa do f1-macro para a Regressão Logística foi: {objeto_de_estudo_lr.best_value}")

Número do melhor trial da Regressão Logística: 12
Parâmetros do melhor trial da Regressão Logística: {'penalidade': 'l1', 'C': 137.97053160889004, 'normalizar': True, 'tipo_norm': 'MinMax'}
A melhor estimativa do f1-macro para a Regressão Logística foi: 0.7383435156883571


### **Conclusão sobre esse notebook**

Neste notebook, implementamos e otimizamos diversos modelos de aprendizado de máquina para prever a toxicidade de nanopartículas com base em suas características físico-químicas e condições experimentais. Utilizamos técnicas como agrupamento de dados para evitar vazamento, codificação one-hot para variáveis categóricas e validação cruzada para garantir a qualidade das métricas.

Os modelos foram otimizados com o Optuna, que permitiu encontrar combinações de hiperparâmetros que maximizassem a estimativa do F1-score macro.

| Modelo               |   F1-Macro |   Melhor Trial |
|----------------------|------------|----------------|
| Baseline             |     0.4339 |              - |
| K-NN                 |     0.8402 |            148 |
| Árvore de Decisão    |     0.8961 |            781 |
| SVC                  |     0.8508 |           1059 |
| Floresta Aleatória   |     0.9112 |            898 |
| Regressão Logística  |     0.7383 |             12 |

A principio foi possível observar que:

Todos os modelos superaram significativamente o baseline, mostrando que as features utilizadas possuem um valor preditivo. Além disso, modelos como Floresta Aleatória e a Árvore de Decisão tiveram um melhor resultado.

O avaliação desses modelos nos dados de teste pode ser vista no arquivo **``notebooks\avaliacao_modelos.ipynb``**